In [2]:
import hail as hl
hl.init()

Running on Apache Spark version 2.4.1
SparkUI available at http://192.168.0.4:4041
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.59-63cf625e29e5
LOGGING: writing to /Users/myohanne/Desktop/Broad/tj/hail_practice/hail-20210122-1122-0.2.59-63cf625e29e5.log


### Errors and notes:

#### 1) downloaded one vcf file locally and tried to import it:

hl.import_vcf("sc_global_japan_rik_Huang_Yoshikawa_schizophrenia_exome-sharded_vcf-sc_global_japan_rik_Huang_Yoshikawa_schizophrenia_exome.filtered.0.vcf.gz.vcf", force =True)

##### Error summary: "MalformedInputException: Input length = 1" - not sure how to fix this or what the error means 

#### 2) import and write the vcf file using its url: 

hl.import_vcf("gs://fc-7a86ab95-f5c5-4ac1-976b-e57dbc601eb1/sc_global_japan_rik_Huang_Yoshikawa_schizophrenia_exome/sharded_vcf/sc_global_japan_rik_Huang_Yoshikawa_schizophrenia_exome.filtered.0.vcf.gz", force=True).write("data1.mt", overwrite = True)

##### Error summary: "HailException: Invalid locus 'chr1:12807' found. Contig 'chr1' is not in the reference genome 'GRCh37'." - changed the default genome from GRCh37 to GRCh38 and that seemed to work

#### Side note: I switched "force=True" with "force_bgz=True" because if I use "force", operations I run downstream (like filter_rows(), annotate_cols() ... ) will be run in a linear fashion, one by one - runs slower. Whereas, if I use "force_bgz" this is not the case, and operations can be parallelized, so they can run multiple operations at the same time

In [13]:
# import vcf file and write as a matrixTable - writes it locally 

hl.import_vcf("gs://fc-7a86ab95-f5c5-4ac1-976b-e57dbc601eb1/sc_global_japan_rik_Huang_Yoshikawa_schizophrenia_exome/sharded_vcf/sc_global_japan_rik_Huang_Yoshikawa_schizophrenia_exome.filtered.0.vcf.gz", force_bgz=True, reference_genome='GRCh38').write("data1.mt", overwrite = True)

2021-01-22 11:42:15 Hail: INFO: Coerced sorted dataset
2021-01-22 11:42:33 Hail: INFO: wrote matrix table with 12979 rows and 1279 columns in 1 partition to data1.mt
    Total size: 57.09 MiB
    * Rows/entries: 57.09 MiB
    * Columns: 5.03 KiB
    * Globals: 11.00 B
    * Smallest partition: 12979 rows (57.09 MiB)
    * Largest partition:  12979 rows (57.09 MiB)


In [14]:
# read written file and assign it to a variable 

mt = hl.read_matrix_table('data1.mt')

### Notes:

In hail, Tables = row fields and globals whereas matrix table = a table with an extra dimension: row fields, column fields, entry fields and globals

Column fields - info about sample phenotypes, ancestry, sex, and covariates 

Row fields - info like gene membership and functional impact for use in QC or analysis

Entry fields - there is an entry for each (row, column) pair

In [82]:
# matrixTable description  

mt.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
----------------------------------------
Row fields:
    'locus': locus<GRCh38>
    'alleles': array<str>
    'rsid': str
    'qual': float64
    'filters': set<str>
    'info': struct {
        AC: array<int32>, 
        AF: array<float64>, 
        AN: int32, 
        AS_BaseQRankSum: array<float64>, 
        AS_FS: array<float64>, 
        AS_InbreedingCoeff: array<float64>, 
        AS_MQ: array<float64>, 
        AS_MQRankSum: array<float64>, 
        AS_QD: array<float64>, 
        AS_RAW_BaseQRankSum: str, 
        AS_RAW_MQ: str, 
        AS_RAW_MQRankSum: str, 
        AS_RAW_ReadPosRankSum: str, 
        AS_ReadPosRankSum: array<float64>, 
        AS_SB_TABLE: str, 
        AS_SOR: array<float64>, 
        BaseQRankSum: float64, 
        DB: bool, 
        DP: int32, 
        DS: bool, 
        END: int32, 
        ExcessHet: float64, 
        

In [83]:
# number of columns in the matrixTable 
mt.count_cols()

1279

In [84]:
# number of rows in the matrixTable 
mt.count_rows()

12979

In [85]:
# returns both rows and columns 
mt.count()

(12979, 1279)

In [86]:
# show all row fields of the first 5 varients 

mt.rows().show(5)

+---------------+---------------+------+----------+---------------------------------+--------------+----------------+---------+----------------------+----------------+-------------------------+
| locus         | alleles       | rsid |     qual | filters                         | info.AC      | info.AF        | info.AN | info.AS_BaseQRankSum | info.AS_FS     | info.AS_InbreedingCoeff |
+---------------+---------------+------+----------+---------------------------------+--------------+----------------+---------+----------------------+----------------+-------------------------+
| locus<GRCh38> | array<str>    | str  |  float64 | set<str>                        | array<int32> | array<float64> |   int32 | array<float64>       | array<float64> | array<float64>          |
+---------------+---------------+------+----------+---------------------------------+--------------+----------------+---------+----------------------+----------------+-------------------------+
| chr1:12807    | ["C","T"]     | NA   | 5.16e+03 | {"VQSRTrancheSNP99.95to100.00"} | [22]         | [3.67e-01]     |      60 | [0.00e+00]           | [0.00e+00]     | [1.40e-01]              |
| chr1:13054    | ["C","T"]     | NA   | 5.35e+02 | {"VQSRTrancheSNP99.95to100.00"} | [9]          | [3.50e-02]     |     254 | [0.00e+00]           | [0.00e+00]     | [4.03e-01]              |
| chr1:13079    | ["C","G"]     | NA   | 1.48e+03 | {"VQSRTrancheSNP99.95to100.00"} | [18]         | [8.70e-02]     |     208 | [0.00e+00]           | [1.24e+01]     | [1.15e-01]              |
| chr1:13273    | ["G","C"]     | NA   | 4.75e+03 | {"VQSRTrancheSNP99.80to99.90"}  | [10]         | [6.25e-01]     |      16 | NA                   | [0.00e+00]     | NA                      |
| chr1:13417    | ["C","CGAGA"] | NA   | 2.53e+03 | {}                              | [4]          | [6.67e-01]     |       6 | NA                   | [0.00e+00]     | NA                      |
+---------------+---------------+------+----------+---------------------------------+--------------+----------------+---------+----------------------+----------------+-------------------------+

+----------------+-------------------+----------------+--------------------------+----------------+-----------------------+----------------------------+------------------------+------------------+
| info.AS_MQ     | info.AS_MQRankSum | info.AS_QD     | info.AS_RAW_BaseQRankSum | info.AS_RAW_MQ | info.AS_RAW_MQRankSum | info.AS_RAW_ReadPosRankSum | info.AS_ReadPosRankSum | info.AS_SB_TABLE |
+----------------+-------------------+----------------+--------------------------+----------------+-----------------------+----------------------------+------------------------+------------------+
| array<float64> | array<float64>    | array<float64> | str                      | str            | str                   | str                        | array<float64>         | str              |
+----------------+-------------------+----------------+--------------------------+----------------+-----------------------+----------------------------+------------------------+------------------+
| [2.47e+01]     | [-6.00e-01]       | [2.54e+01]     | NA                       | NA             | NA                    | NA                         | [1.05e+00]             | NA               |
| [2.91e+01]     | [1.00e+00]        | [2.97e+01]     | NA                       | NA             | NA                    | NA                         | [1.60e+00]             | NA               |
| [2.46e+01]     | [-4.00e-01]       | [3.30e+01]     | NA                       | NA             | NA                    | NA                         | [-7.50e-01]            | NA               |
| [3.04e+01]     | NA                | [3.10e+01]     | NA                       | NA             | NA                    | NA                         | NA                     | NA               |
| [3.70e+01]     | NA                | [2.82e+01]     | NA                       | NA       

In [87]:
# when select() is empty, it only shows the row key fileds. But when you specify a field name inside select(), 
# then it will show the row key fields and that specified row field 

mt.rows().select("rsid").show(5)

,,
locus,alleles,rsid
locus<GRCh38>,array<str>,str
chr1:12807,"[""C"",""T""]",NA
chr1:13054,"[""C"",""T""]",NA
chr1:13079,"[""C"",""G""]",NA
chr1:13273,"[""G"",""C""]",NA
chr1:13417,"[""C"",""CGAGA""]",NA


In [88]:
# the following lines of code have the same outputs. They both print the key fields for the top 5 rows  

mt.row_key.show(5)
mt.rows().select().show(5)

,
locus,alleles
locus<GRCh38>,array<str>
chr1:12807,"[""C"",""T""]"
chr1:13054,"[""C"",""T""]"
chr1:13079,"[""C"",""G""]"
chr1:13273,"[""G"",""C""]"
chr1:13417,"[""C"",""CGAGA""]"


,
locus,alleles
locus<GRCh38>,array<str>
chr1:12807,"[""C"",""T""]"
chr1:13054,"[""C"",""T""]"
chr1:13079,"[""C"",""G""]"
chr1:13273,"[""G"",""C""]"
chr1:13417,"[""C"",""CGAGA""]"


In [89]:
# description of the column - we can see that there is only one column (sample) in our matrixTable  

mt.col.describe()

# print column schema
#print(mt.col.dtype)

--------------------------------------------------------
Type:
        struct {
        s: str
    }
--------------------------------------------------------
Source:
Index:
    ['column']
--------------------------------------------------------


In [90]:
# first 5 sample IDs 

mt.s.show(5)

""
s
str
"""JP-RIK-C-00070"""
"""JP-RIK-C-00071"""
"""JP-RIK-C-00072"""
"""JP-RIK-C-00073"""
"""JP-RIK-C-00074"""


In [91]:
# first 5 genotype calls - all entry fields for each sample
# take() - rows as a list  
# show() - rows in a table format (includes row key fields too)

mt.entry.show(5)
mt.entry.take(5)

+---------------+---------------+---------------------+---------------------+---------------------+---------------------+-------------------------+----------------------+----------------------+
| locus         | alleles       | 'JP-RIK-C-00070'.AD | 'JP-RIK-C-00070'.DP | 'JP-RIK-C-00070'.GQ | 'JP-RIK-C-00070'.GT | 'JP-RIK-C-00070'.MIN_DP | 'JP-RIK-C-00070'.PGT | 'JP-RIK-C-00070'.PID |
+---------------+---------------+---------------------+---------------------+---------------------+---------------------+-------------------------+----------------------+----------------------+
| locus<GRCh38> | array<str>    | array<int32>        |               int32 |               int32 | call                |                   int32 | call                 | str                  |
+---------------+---------------+---------------------+---------------------+---------------------+---------------------+-------------------------+----------------------+----------------------+
| chr1:12807    | ["C","T"]     | [0,0]               |                   0 |                  NA | NA                  |                      NA | NA                   | NA                   |
| chr1:13054    | ["C","T"]     | [4,0]               |                   4 |                  12 | 0/0                 |                      NA | NA                   | NA                   |
| chr1:13079    | ["C","G"]     | [0,0]               |                   0 |                  NA | NA                  |                      NA | NA                   | NA                   |
| chr1:13273    | ["G","C"]     | [0,0]               |                   0 |                  NA | NA                  |                      NA | NA                   | NA                   |
| chr1:13417    | ["C","CGAGA"] | [0,0]               |                   0 |                  NA | NA                  |                      NA | NA                   | NA                   |
+---------------+---------------+---------------------+---------------------+---------------------+---------------------+-------------------------+----------------------+----------------------+

+---------------------+----------------------+---------------------+
| 'JP-RIK-C-00070'.PL | 'JP-RIK-C-00070'.RGQ | 'JP-RIK-C-00070'.SB |
+---------------------+----------------------+---------------------+
| array<int32>        |                int32 | array<int32>        |
+---------------------+----------------------+---------------------+
| [0,0,0]             |                   NA | NA                  |
| [0,12,119]          |                   NA | NA                  |
| [0,0,0]             |                   NA | NA                  |
| [0,0,0]             |                   NA | NA                  |
| [0,0,0]             |                   NA | NA                  |
+---------------------+----------------------+---------------------+
showing top 5 rows
showing the first 1 of 1279 columns

[Struct(AD=[0, 0], DP=0, GQ=None, GT=None, MIN_DP=None, PGT=None, PID=None, PL=[0, 0, 0], RGQ=None, SB=None),
 Struct(AD=[0, 0], DP=0, GQ=None, GT=None, MIN_DP=None, PGT=None, PID=None, PL=[0, 0, 0], RGQ=None, SB=None),
 Struct(AD=[0, 0], DP=0, GQ=None, GT=None, MIN_DP=None, PGT=None, PID=None, PL=[0, 0, 0], RGQ=None, SB=None),
 Struct(AD=[0, 0], DP=0, GQ=None, GT=None, MIN_DP=None, PGT=None, PID=None, PL=[0, 0, 0], RGQ=None, SB=None),
 Struct(AD=[0, 0], DP=0, GQ=None, GT=None, MIN_DP=None, PGT=None, PID=None, PL=[0, 0, 0], RGQ=None, SB=None)]

In [92]:
# call rates at the first 5 genotypes - only shows "GT" entry field for each sample  

mt.GT.show(5)

,,,,,,,,,,,,,,,,
,,'JP-RIK-C-00070','JP-RIK-C-00071','JP-RIK-C-00072','JP-RIK-C-00073','JP-RIK-C-00074','JP-RIK-C-00075','JP-RIK-C-00076','JP-RIK-C-00077','JP-RIK-C-00078','JP-RIK-C-00079','JP-RIK-C-00080','JP-RIK-C-00081','JP-RIK-C-00082','JP-RIK-C-00083','JP-RIK-C-00084'
locus,alleles,GT,GT,GT,GT,GT,GT,GT,GT,GT,GT,GT,GT,GT,GT,GT
locus<GRCh38>,array<str>,call,call,call,call,call,call,call,call,call,call,call,call,call,call,call
chr1:12807,"[""C"",""T""]",NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,1/1,NA
chr1:13054,"[""C"",""T""]",0/0,NA,0/0,NA,NA,NA,NA,NA,0/0,NA,NA,NA,NA,NA,NA
chr1:13079,"[""C"",""G""]",NA,NA,0/0,NA,NA,NA,NA,NA,0/0,NA,NA,NA,NA,1/1,NA
chr1:13273,"[""G"",""C""]",NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
chr1:13417,"[""C"",""CGAGA""]",NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [93]:
# distribution of genotype calls

#mt.aggregate_entries(hl.agg.counter(mt.GT.n_alt_alleles()))

In [94]:
# the sample_qc function in hail produces a set of useful metrics and stores them in a column field

(hl.sample_qc(mt)).col.describe()

--------------------------------------------------------
Type:
        struct {
        s: str, 
        sample_qc: struct {
            dp_stats: struct {
                mean: float64, 
                stdev: float64, 
                min: float64, 
                max: float64
            }, 
            gq_stats: struct {
                mean: float64, 
                stdev: float64, 
                min: float64, 
                max: float64
            }, 
            call_rate: float64, 
            n_called: int64, 
            n_not_called: int64, 
            n_filtered: int64, 
            n_hom_ref: int64, 
            n_het: int64, 
            n_hom_var: int64, 
            n_non_ref: int64, 
            n_singleton: int64, 
            n_snp: int64, 
            n_insertion: int64, 
            n_deletion: int64, 
            n_transition: int64, 
            n_transversion: int64, 
            n_star: int64, 
            r_ti_tv: float64, 
            r_het_hom_var: f